In [1]:
import pyspark
from pyspark.sql.functions import *
spark = pyspark.sql.SparkSession.builder.getOrCreate()

22/05/23 11:15:12 WARN Utils: Your hostname, Brandyns-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.3.168 instead (on interface en0)
22/05/23 11:15:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/23 11:15:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read the case, department, and source data into their own spark dataframes.

In [2]:
case = spark.read.csv('case.csv', sep=",", header=True, inferSchema=True)

In [3]:
department = spark.read.csv('dept.csv', sep=",", header=True, inferSchema=True)

In [4]:
source = spark.read.csv('source.csv', sep=",", header=True, inferSchema=True)

Store the source data in both csv and json format

In [5]:
source.write.json('source_json', mode='overwrite')
source.write.csv('source_csv', mode='overwrite')

Inspect the folder structure, what do you notice?

- The above are not files but directories

Inspect the data in the dataframes, are the data types appropriate? Write code necessary to cast the values to the appropriate types

In [6]:
# Showing the first row of case
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [7]:
# Showing case dtypes
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [8]:
# Rename SLA_due_date column to better match other date-type columns
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

In [9]:
# Columns case_closed and case_late store yes/no values, they are currently strings and should be converted to boolean
# Confirming they are just yes/no
case.groupby('case_closed', 'case_late').count().show()

+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [10]:
# Changing case_closed and case_late to boolean
case = (case.withColumn("case_closed", expr('case_closed == "Yes"')).withColumn("case_late", 
                                                                               expr('case_late == "Yes"')))

In [13]:
# Changing council_district to string
case = case.withColumn("council_district", col('council_district').cast('string'))

In [15]:
# Dealing with timestamp changes
# Establishing our date format
fmt = "M/d/yy H:mm"

In [16]:
case = (
case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))
    .withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))
    .withColumn('case_due_date', to_timestamp('case_due_date', fmt))

)